In [1]:
# load required dependencies
%pip install --upgrade pip
%pip install -r "./requirements.txt"





Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [42]:
# import required library modules
import requests
import pprint 
import csv
import re
from bs4 import BeautifulSoup

def try_parse_int(s, base=10, val=None):
  try:
    return int(s, base)
  except ValueError:
    return val

# read the GoTriple Keywords analysis Google sheet
html = requests.get('https://docs.google.com/spreadsheets/d/1rI_BrE6BcyWCkipaKqTZ5bQJXcLwFMMkLpLMu1s7gKg').text
soup = BeautifulSoup(html, "lxml")
tables = soup.find_all("table")
tablerows = tables[0].find_all("tr")
items = rows_to_items(tablerows)

def row_to_item(row): 
    cols = row.find_all("td")
    keyword = (cols[0].text if len(cols) > 0 else "").strip()
    counter = try_parse_int((cols[1].text if len(cols) > 1 else "").strip())
    return { 
        "keyword": keyword, 
        "counter": counter if isinstance(counter, int) else 0 
    }

def rows_to_items(rows):
    items = map(lambda row: row_to_item(row), rows)
    return list(filter(lambda item: item.get("counter", 0) > 0, items))

def filter_by_regex(items: list=[], pattern: str="^.*$"):
    reg = re.compile(pattern)  
    return list(filter(lambda item: reg.fullmatch(item.get("keyword")), items))

def categorize_by_regex(items: list=[], pattern: str="^.*$", category: str=""):
    matches = filter_by_regex(items, pattern)
    for item in matches:
        item["category"] = category
    return matches

# categorizing the patterns identified in the keywords
# by categorizing first, we can then treat each keyword appropriately
categorize(items, r'^ddc\:\d{3}$', "DDC_NUMBER")
categorize(items, r'^[A-Z]{1,2}\d+\-\d+$', "HYPHENATED_CODE")
categorize(items, r'^[A-Z]$', "SINGLE_UPPERCASE_LETTER")
categorize(items, r'^[A-Z\s]{2,}$', "ALL_UPPERCASE")
categorize(items, r'^\([A-Z]+\-[A-Z]+\).*$', "PREFIXED_HYPHENATED_CODE")
categorize(items, r'^\[[A-Z]+\.[A-Z]+\].*$', "PREFIXED_DOTTED_CODE")
categorize(items, r'^\[[A-Z]+\]\w+$', "XXX")
#reg = re.compile(r'^ddc\:\d{3}$')                    # Compile the regex
#lst = list(filter(lambda item: reg.fullmatch(item.get("keyword")), items))

pprint.pprint(items)


[{'category': 'DDC_NUMBER', 'counter': 505064, 'keyword': 'ddc:330'},
 {'counter': 196710, 'keyword': 'Education'},
 {'counter': 185533, 'keyword': 'Economics'},
 {'counter': 182100, 'keyword': '(Produktform)Paperback / softback'},
 {'counter': 160773, 'keyword': 'Article'},
 {'counter': 158862, 'keyword': 'Geography and travel'},
 {'counter': 154074, 'keyword': 'Medicine'},
 {'counter': 151138, 'keyword': 'Geography of and travel in Europe'},
 {'counter': 150867, 'keyword': 'General serial publications'},
 {'counter': 150838, 'keyword': 'Psychology'},
 {'counter': 145058,
  'keyword': 'German literature and literatures of related languages'},
 {'category': 'SINGLE_UPPERCASE_LETTER', 'counter': 140464, 'keyword': 'R'},
 {'category': 'PREFIXED_HYPHENATED_CODE',
  'counter': 109361,
  'keyword': '(VLB-PF)BC: Paperback'},
 {'counter': 105915, 'keyword': 'Archaeology'},
 {'counter': 101878, 'keyword': 'Science'},
 {'counter': 100638, 'keyword': 'Social Sciences'},
 {'category': 'SINGLE_UPP